# Analysis Pipeline for Intracranial Data


## STEPS:
##### 1. run the IMPORT AND LOAD DATA section.

##### 2. fill in the values extracted from the plot from step 1 for following variables:


Let's go

In [1]:
import os
import numpy as np
import mne
from mne.utils import _TempDir
import matplotlib.pyplot as plt
import pyqtgraph as pg
import pyqtgraph as plotWidget
%matplotlib qt
from pd_parser.parse_pd import _read_raw, _to_tsv
import time

# DEFINE PATHS

data_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/PAT_3066'
# PT_id = 
script_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/Scripts'
data_raw_file = os.path.join(data_path, 'PAT_3066_EEG_708977_Anon_CategoryLocalizer.edf')
Exper = 'CatLoc'
PT_id = 'PAT_3066'
print(PT_id)

# data_raw_file = os.path.join(data_path, 'PAT_3066_EEG_708977_Anon_CategoryLocalizer.edf') #PAT_3066_EEG_695997_Anon_FLM.edf

# LOAD TRIGGER DATA 
trigger_ch = mne.io.read_raw_edf(data_raw_file,exclude=['ECG+','ECG-','MKR4+','Xe2', 'Xe3', 'Xe4', 'Xe5', 'Xe6'])
print('*   Trigger channel length: ',trigger_ch.__len__)

picks = mne.pick_channels_regexp(trigger_ch.info.ch_names, regexp='Xe1')
fig = plt.figure(figsize=(100, 2))
trigger_ch2 = trigger_ch.get_data(picks=picks)
trigger_ch2 = trigger_ch2[0].T
time_s = np.linspace(0,len(trigger_ch2),num=len(trigger_ch2))

# pg.plot(time_s,trigger_ch2)
# plt.close(fig)


# VISUALIZE
fig = plt.figure(figsize=(100, 2))
print(np.shape(trigger_ch2))
import pyqtgraph as pg
import pyqtgraph as plotWidget
pg.plot(time_s,trigger_ch2)
plt.close(fig)


PAT_3066
Extracting EDF parameters from /Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/PAT_3066/PAT_3066_EEG_708977_Anon_CategoryLocalizer.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
*   Trigger channel length:  <bound method BaseRaw.__len__ of <RawEDF | PAT_3066_EEG_708977_Anon_CategoryLocalizer.edf, 204 x 2181120 (1065.0 s), ~196 kB, data not loaded>>
(2181120,)


### Extract the PD Trigger

In [2]:
from get_trigger_indexes_photodiode import get_trigger_indexes_photodiode
# t_onset_s = input("Enter cut start time: ")
# t_offset_s = input("Enter cut end time: ")
# t_onset_s= 550000
# t_offset_s= 3310000
t_onset_s= 1022000
t_offset_s= 1900000
index_onset,index_offset , trigger_ch_new,_ = get_trigger_indexes_photodiode(data_raw_file = data_raw_file, data_path=data_path,t_start= t_onset_s, t_end=t_offset_s,threshold_val=0.8)

index_on_true=index_onset
index_off_true=index_offset

objc[34104]: Class QCocoaMenuLoader is implemented in both /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/PyQt5/Qt/plugins/platforms/libqcocoa.dylib (0x114c0d430) and /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/cv2/.dylibs/QtGui (0x154a2a2a0). One of the two will be used. Which one is undefined.
objc[34104]: Class QNSApplication is implemented in both /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/PyQt5/Qt/plugins/platforms/libqcocoa.dylib (0x114c0d2f0) and /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/cv2/.dylibs/QtGui (0x154a2a2f0). One of the two will be used. Which one is undefined.
objc[34104]: Class QCocoaApplicationDelegate is implemented in both /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/PyQt5/Qt/plugins/platforms/libqcocoa.dylib (0x114c0d2a0) and /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-pa

*   Data location.... : /var/folders/85/ct7qgj4500s62pw9zxkv_xc40000gn/T/tmp_mne_tempdir_gvl568ok


Extracting EDF parameters from /Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/PAT_3066/PAT_3066_EEG_708977_Anon_CategoryLocalizer.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
878000 (878000,)
*   length of onset indexes:  3
*   length of offset indexes:  130
*   length of Time & Trigger:  878000   878000


### VALIDATION
Assert that the trigger onset and offset are a pair and the onset is before the offset

In [3]:
# VALIDATION
index_on_true=index_onset
index_off_true=index_offset
index_onset = list(index_onset)
index_offset = list(index_offset)
validity = [] # 0 for invalid, 1 for valid
for i in range(min([len(index_on_true),len(index_off_true)])):
    tempe=True
    # print(i)
    # if i >= min([len(index_on_true),len(index_off_true)]):
    #     validity.append(0)
    if index_onset[i] < index_offset[i]:
        while tempe:
            while index_onset[i] < index_offset[i]:
                if index_onset[i+1] <index_offset[i]:
                    # print(index_onset[i])
                    index_onset.pop(i)
                    validity.append(0)
                    tempe=False
                    break
                else:
                    validity.append(1)
                    tempe=False
                    break
    else:
        while tempe:
            while index_onset[i] > index_offset[i]:
                if index_onset[i+1] >index_offset[i]:
                    index_offset.pop(i)
                    validity.append(0)
                    tempte=False
                else:
                    validity.append(1)
                    tempe=False
                    break



# if len(index_offset)!=len(index_onset):
#     if len(index_offset)>len(index_onset):
#         index_onset.append(0)
diff_oo=[index_offset[i]-index_onset[i] for i in range(len(index_offset))]
assert np.min(diff_oo)>=0
# print(validity, len(validity),len(index_onset),len(index_offset))
validity=np.array(validity)

KeyboardInterrupt: 

In [ ]:

# VISUALIZE
fig = plt.figure(figsize=(100, 2))
print(np.shape(trigger_ch2))
import pyqtgraph as pg
import pyqtgraph as plotWidget
# pg.plot(time_s,trigger_ch2)
# pg.plot(time_s,index_onset)
# plt.close(fig)
time_s = np.linspace(0,len(trigger_ch2),num=len(trigger_ch2))
plotWidget = pg.plot(title="Validation of Triggers PD: x is onset, o is offset")
plotWidget.plot(time_s,trigger_ch2)
plotWidget.plot(index_offset,trigger_ch2[index_offset],pen=None, symbol='o')
plotWidget.plot(index_onset,trigger_ch2[index_onset],pen=None, symbol='x')
# plotWidget.plot(index_onset[validity==False],trigger_ch2[index_onset[validity==False]],pen=None, symbol='x')

# plotWidget.show()

### Save The Data

In [ ]:
from datetime import datetime
date_txt=str(datetime.date(datetime.now()))
name_txt = PT_id +Exper+'_triggerPD_' + date_txt+'.txt'
with open(os.path.join(data_path,name_txt ), "w") as textfile:
    textfile.write('onsets;offsets\n')
    XY = [i for i in zip(index_onset, index_offset)]
    for x, y in XY:
        textfile.write(str(x)+';'+str(y))
        textfile.write('\n')
    textfile.close()

    28840
    30446

### FINISHED HERE: 
##### Everything below is draft